In [1]:
import os
import sys
import time
sys.path.insert(0,'..')

from noveltyDetectionConfig import CONFIG
import numpy as np
import itertools
import multiprocessing
from datetime import datetime, timedelta

from NNNoveltyDetectionAnalysis import NNNoveltyDetectionAnalysis

num_processes = multiprocessing.cpu_count()

from Functions.telegrambot import Bot

my_bot = Bot("lisa_thebot")

# Enviroment variables
data_path = CONFIG['OUTPUTDATAPATH']
results_path = CONFIG['PACKAGE_NAME']

training_params = {
    "Technique": "NeuralNetwork",
    "DevelopmentMode": False,
    "DevelopmentEvents": 400,
}
analysis = NNNoveltyDetectionAnalysis(parameters=training_params, 
                                      model_hash="35038f1fdb900e374ada1a03b216d979926b694e68f8ca24cc3d74914e8c1f37", 
                                      load_hash=True, load_data=True, verbose=True)
all_data, all_trgt, all_trgt_sparse = analysis.getData()

trn_data = analysis.trn_data
trn_trgt = analysis.trn_trgt
trn_trgt_sparse = analysis.trn_trgt_sparse

models = analysis.models

Using TensorFlow backend.


Reading from /home/vinicius.mello/Workspace/SonarAnalysis/Results/NoveltyDetection/NeuralNetwork/outputs/35038f1fdb900e374ada1a03b216d979926b694e68f8ca24cc3d74914e8c1f37/parameters.json
[+] Time to read data file: 1.39137101173 seconds
Qtd event of A is 12939
Qtd event of B is 29352
Qtd event of C is 11510
Qtd event of D is 23760

Biggest class is B with 29352 events
Total of events in the dataset is 77561
Balacing data...
DataHandler Class: CreateEventsForClass
Original Size: (12939, 400)
DataHandler Class: CreateEventsForClass
Original Size: (29352, 400)
DataHandler Class: CreateEventsForClass
Original Size: (11510, 400)
DataHandler Class: CreateEventsForClass
Original Size: (23760, 400)
Reading from /home/vinicius.mello/Workspace/SonarAnalysis/Results/NoveltyDetection/4_folds_cross_validation_balanced_data.jbl


In [3]:
%matplotlib inline 

from sklearn import metrics
from sklearn import preprocessing
from sklearn.externals import joblib
import matplotlib.pyplot as plt

# Choose layer 
layer = 1

# Choose neurons topology
hidden_neurons = [50]

step = 5
neurons_mat = [1] + range(step,hidden_neurons[layer-1]+step,step)
neurons_mat = neurons_mat[:len(neurons_mat)-layer+2]

analysis_name = 'feature_relevance_analysis_%i_layer'%(layer)
analysis_file = os.path.join(analysis.getBaseResultsPath(), "AnalysisFiles", analysis_name + ".jbl")    

verbose = True

# Plot parameters
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.fontsize'] = 14
m_colors = ['b', 'r', 'g', 'y']
figsize = (10,5)

n_bins = all_data.shape[1]
sp_per_bin = np.zeros(n_bins)
for ibin in range(n_bins):
    temp_all_data = all_data[:,:]
    temp_all_data[:,ibin] = 0
    
    #Get SP